In [ ]:
import pandas as pd
from datetime import datetime

df_train = pd.read_csv('douban_train.csv')
df_test = pd.read_csv('douban_test.csv')

df_train['UserID'] = df_train['UserID']
df_test['UserID'] = df_test['UserID']
USERS = max(df_train['UserID'].max(), df_test['UserID'].max()) + 1
df_train['MovieID'] = df_train['MovieID']
df_test['MovieID'] = df_test['MovieID']
MOVIES = max(df_train['MovieID'].max(), df_test['MovieID'].max()) + 1

print(USERS, MOVIES, df_train['MovieID'].max(), df_test['MovieID'].max())

df = pd.concat([df_train, df_test])
ROWS = df['UserID']
COLS = df['MovieID']

print('Finished feature engineering...')
print('df shape', len(df))
print('df_train.head()', len(df_train))
print(df_train.head())
print('df_test.head()', len(df_test))
print(df_test.head())

11777 20697 20696 20681
Finished feature engineering...
df shape 190590
df_train.head() 152528
   Unnamed: 0  Unnamed: 0.1  UserID  MovieID  Rating
0           0             0       0        0       5
1           2             2       0        2       5
2           3             3       0        3       4
3           4             4       0        4       4
4           5             5       0        5       4
df_test.head() 38062
   Unnamed: 0  Unnamed: 0.1  UserID  MovieID  Rating
0           1             1       0        1       5
1          12            12       1       12       3
2          21            21       2       21       5
3          23            23       3       23       5
4          25            25       3       25       5


In [ ]:
import pandas as pd
import numpy as np
from tqdm import trange
from scipy.sparse import csr_matrix
import torch
import random

seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
torch.set_default_tensor_type(torch.FloatTensor)

T = len(df_test)
T_train = len(df_train)

df_test_2 = df_test.copy()
df_test_2['Rating'] = -1

df_train_2 = df_train.copy()
df_train_2['Rating'] = -1

rating_matrix = pd.concat([df_train, df_test_2]).pivot(index='UserID', columns='MovieID', values='Rating')
rating_matrix[rating_matrix.isna()] = -1
rating_matrix = torch.from_numpy(rating_matrix.values).to(device)
non_zero_mask = (rating_matrix != -1)
print(rating_matrix.shape)
USERS,MOVIES = rating_matrix.shape

test_matrix = pd.concat([df_test, df_train_2]).pivot(index='UserID', columns='MovieID', values='Rating')
test_matrix[test_matrix.isna()] = -1
test_matrix = torch.from_numpy(test_matrix.values).to(device)
non_zero_mask_test = (test_matrix != -1)
print(test_matrix.device)

R_train = csr_matrix((df_train['Rating'], (df_train['UserID'], df_train['MovieID'])), shape=(USERS, MOVIES), dtype=np.float)

R_train_tensor = []
for i in range(USERS):
    batch = R_train[i].nonzero()[1]
    ts = torch.from_numpy(R_train[i, batch].todense().transpose().astype(np.float32)).to(device)
    R_train_tensor.append(ts)
print(len(R_train_tensor), R_train_tensor[0].shape)
print(R_train.shape)

R_train_T = R_train.transpose()
R_train_tensor_T = []
for j in range(MOVIES):
    batch = R_train_T[j].nonzero()[1]
    ts = torch.from_numpy(R_train_T[j, batch].todense().transpose().astype(np.float32)).to(device)
    R_train_tensor_T.append(ts)
print(len(R_train_tensor_T), R_train_tensor_T[0].shape)
R_test = csr_matrix((df_test['Rating'], (df_test['UserID'], df_test['MovieID'])), shape=(USERS, MOVIES))

torch.Size([11777, 20697])
cpu
11777 torch.Size([7, 1])
(11777, 20697)
20697 torch.Size([108, 1])


In [ ]:
import time
from scipy.stats import multivariate_normal, wishart
import tensorflow as tf

f = 10
num_epochs = 10

T = len(df_test)
T_train = len(df_train)
U_history = []
V_history = []
U = torch.normal(mean=0., std=0.1, size=(USERS, f))
U_sm175 = torch.zeros(size=(USERS, f))
V_sm175 = torch.zeros(size=(MOVIES, f))

U_sm200 = torch.zeros(size=(USERS, f))
V_sm200 = torch.zeros(size=(MOVIES, f))

U_sm225 = torch.zeros(size=(USERS, f))
V_sm225 = torch.zeros(size=(MOVIES, f))

V = torch.normal(mean=0., std=0.1, size=(MOVIES, f))
R_hat_sm = torch.zeros((USERS, MOVIES))

alpha = 1

v_0 = f
v_users = v_0 + USERS
v_movies = v_0 + MOVIES
W_0 = torch.eye(f)

mean_0 = torch.zeros((f))
beta_0 = 1
beta_users = beta_0 + USERS
beta_movies = beta_0 + MOVIES


for epoch in range(num_epochs):
    t = time.time()
    u_conv_all = torch.stack([torch.einsum('n,m->nm', ui, ui.T) for ui in U])
    v_conv_all = torch.stack([torch.einsum('n,m->nm', vj, vj.T) for vj in V])
    # Compute users variables

    U_mean = torch.mean(U, axis=0, keepdims=True)
    S_U = torch.mean(u_conv_all, axis=0)

    W_users = torch.inverse(W_0 + USERS * S_U + beta_0 * USERS / (beta_0 + USERS) * (mean_0 - U_mean).T @ (mean_0 - U_mean))
    mean_0_star = torch.squeeze(((beta_0 * mean_0 + USERS * U_mean) / (beta_0 + USERS))).to(dtype=torch.float64)

    # Sample user precision and mean


    precision_u = torch.from_numpy(wishart.rvs(v_users, scale=W_users))
    # print((beta_users * covariance_u).dtype)
    # print(mean_0_star.dtype)
    # mean_u = torch.from_numpy(multivariate_normal.rvs(mean_0_star, cov=torch.inverse(beta_users * covariance_u))).reshape((1, f))
    mean_u = torch.unsqueeze(torch.distributions.MultivariateNormal(loc=mean_0_star, precision_matrix=beta_users * precision_u).sample(), 0)


    # Sample user feature matrix
    for i in range(USERS):
      batch = torch.from_numpy(R_train[i].nonzero()[1]).to(dtype=torch.int64)
      v_sum = torch.sum(v_conv_all[batch], axis=0)
      v_conv = torch.sum(R_train_tensor[i] * V[batch], axis=0)      
      
      precision_i = precision_u + alpha * v_sum
      covariance_i = torch.inverse(precision_i)
      mean_i = torch.squeeze(((alpha * v_conv + mean_u @ precision_u) @ covariance_i))
      # U[i] = torch.from_numpy(multivariate_normal.rvs(mean_i, cov=inv_covariance_i))
      U[i] = torch.distributions.MultivariateNormal(mean_i, covariance_i).sample()
    # Compute movies variables

    V_mean = torch.mean(V, axis=0, keepdims=True)
    S_V = torch.mean(v_conv_all, axis=0)
    
    W_movies = torch.inverse(W_0 + MOVIES * S_V + beta_0 * MOVIES / (beta_0 + MOVIES) * (mean_0 - V_mean).T @ (mean_0 - V_mean))
    mean_0_star = torch.squeeze(((beta_0 * mean_0 + MOVIES * V_mean) / (beta_0 + MOVIES))).to(dtype=torch.float64)
    
    # Sample movie covariance and mean

    precision_v = torch.from_numpy(wishart.rvs(v_movies, scale=W_movies))
    # mean_v = torch.from_numpy(multivariate_normal.rvs(mean_0_star, cov=torch.inverse(beta_movies * covariance_v))).reshape((1, f))
    mean_v = torch.unsqueeze(torch.distributions.MultivariateNormal(loc=mean_0_star, precision_matrix=beta_movies * precision_v).sample(), 0)

    u_conv_all = torch.stack([torch.einsum('n,m->nm', ui, ui.T) for ui in U])
    
    # Sample movie feature matrix

    for j in range(MOVIES):
      batch = torch.from_numpy(R_train_T[j].nonzero()[1]).to(dtype=torch.int64)
      u_sum = torch.sum(u_conv_all[batch], axis=0)
      u_conv = torch.sum(R_train_tensor_T[j] * U[batch], axis=0)

      precision_j = precision_v + alpha * u_sum
      covariance_j = torch.inverse(precision_j)
      mean_j = torch.squeeze(((alpha * u_conv + mean_v @ precision_v) @ covariance_j))

      V[j] = torch.distributions.MultivariateNormal(mean_j, covariance_j).sample()
      # V[j] = torch.from_numpy(multivariate_normal.rvs(mean_j, cov=inv_covariance_j))


    # R_hat = U @ V.T
    if t > 175:
      U_sm175 = U_sm175 + U
      V_sm175 = V_sm175 + V
    
    if t > 200:
      U_sm200 = U_sm200 + U
      V_sm200 = V_sm200 + V

    if t > 225:
      U_sm225 = U_sm225 + U
      V_sm225 = V_sm225 + V
    print('Epoch', epoch, time.time() - t)

torch.save(U_sm175, 'U175.pt')
torch.save(V_sm175, 'V175.pt')
torch.save(U_sm200, 'U200.pt')
torch.save(V_sm200, 'V200.pt')
torch.save(U_sm225, 'U225.pt')
torch.save(V_sm225, 'V225.pt')

Epoch 0 42.57160186767578
Epoch 1 42.52551031112671
Epoch 2 42.29231405258179
Epoch 3 41.80431938171387
Epoch 4 42.72337794303894
Epoch 5 43.36680746078491
Epoch 6 42.1110737323761
Epoch 7 41.75307273864746
Epoch 8 42.25348901748657
Epoch 9 42.37766981124878
